# Classification

**CS5483 Data Warehousing and Data Mining**
___

In [1]:
load("datamining.mac")$

## Decision tree induction

### Distribution

`empirical(lst)` computes the empirical distribution of the list `lst`.

In [2]:
block(
    [lst:[2,2,1,3,3,3]],
    empirical(lst)
);

                                         1  1  1
(%o1)                       [[1, 2, 3], [-, -, -]]
                                         6  3  2

A pair is returned, where 
- the first element is the list of unique values sorted in ascending order, and 
- the second element is their fractional number of occurences.

### Information gain

An impurity measure for decision tree induction is entropy computed as `entropy(ps)` for some distribution `ps` as a list of probability masses:

In [3]:
entropy(ps);

               ====
               \
(%o2)           >      (if equal(p, 0) then 0 else - p log2(p))
               /
               ====
               p in ps

The information gain ratios and related information quantities can be computed as follows:

In [4]:
block(
    [
        fns: ['i, 'X_1, 'X_2, 'Y],
        n: 6,
        gen: lambda([i], [i, random(2), random(2), random(2)]),
        conds: ['X_1, 'X_2],
        target: 'Y,
        data, info
    ],
    data: build_data(fns, gen, n),
    [
        data,
        Info(data, target),
        build_data_from_list(
            ['X, 'Info[X], 'Gain[X], 'SplitInfo[X], 'GainRatio[X]],
            makelist(
                map('simplify,
                    [X,
                     InfoX(data, target, X), 
                     Gain(data, target, X), 
                     SplitInfo(data, X), 
                     GainRatio(data, target, X)]
                ), 
                X, conds
            )
        )
    ]
);

       [ i  X_1  X_2  Y ]
       [                ]
       [ 1   0    0   0 ]
       [                ]
       [ 2   1    0   1 ]
       [                ]
(%o3) [[ 3   1    1   0 ], 1, matrix([X, Info , Gain , SplitInfo , 
       [                ]                    X      X           X
       [ 4   1    1   1 ]
       [                ]
       [ 5   0    0   1 ]
       [                ]
       [ 6   0    1   0 ]
                                  2                               2
                   log(3) - 2 log(-)    log(3) - 3 log(2) - 2 log(-)
                                  3                               3
GainRatio ], [X_1, -----------------, - ----------------------------, 1, 
         X             3 log(2)                   3 log(2)
                            2                         2
  log(3) - 3 log(2) - 2 log(-)         log(3) - 2 log(-)
                            3                         3
- ----------------------------], [X_2, -----------------, 
            3 

where

- `Info(data, target)` computes the information content (entropy) of `target` in `data`.
- `InfoX(data, target, X)` computes the information (conditional entropy) given `X`.
- `Gain(data, target, X)` calculates the information gain of `target` with `X`.
- `SplitInfo(data, X)` calculates the split information (entropy) of `X`.
- `GainRatio(data, target, X)` calculates the information gain ratio of `target` with `X`.

In [5]:
describe(makelist)$


 -- Function: makelist
          makelist ()
          makelist (<expr>, <n>)
          makelist (<expr>, <i>, <i_max>)
          makelist (<expr>, <i>, <i_0>, <i_max>)
          makelist (<expr>, <i>, <i_0>, <i_max>, <step>)
          makelist (<expr>, <x>, <list>)

     The first form, 'makelist ()', creates an empty list.  The second
     form, 'makelist (<expr>)', creates a list with <expr> as its single
     element.  'makelist (<expr>, <n>)' creates a list of <n> elements
     generated from <expr>.

     The most general form, 'makelist (<expr>, <i>, <i_0>, <i_max>,
     <step>)', returns the list of elements obtained when 'ev (<expr>,
     <i>=<j>)' is applied to the elements <j> of the sequence: <i_0>,
     <i_0> + <step>, <i_0> + 2*<step>, ..., with <|j|> less than or
     equal to <|i_max|>.

     The increment <step> can be a number (positive or negative) or an
     expression.  If it is omitted, the default value 1 will be used.
     If both <i_0> and <step> are omitted, 

In [6]:
describe(map)$




  There are also some inexact matches for `map'.
  Try `?? map' to see them.




 -- Function: map (<f>, <expr_1>, ..., <expr_n>)

     Returns an expression whose leading operator is the same as that of
     the expressions <expr_1>, ..., <expr_n> but whose subparts are the
     results of applying <f> to the corresponding subparts of the
     expressions.  <f> is either the name of a function of n arguments
     or is a 'lambda' form of n arguments.

     'maperror' - if 'false' will cause all of the mapping functions to
     (1) stop when they finish going down the shortest <expr_i> if not
     all of the <expr_i> are of the same length and (2) apply <f> to
     [<expr_1>, <expr_2>, ...] if the <expr_i> are not all the same type
     of object.  If 'maperror' is 'true' then an error message will be
     given in the above two instances.

     One of the uses of this function is to 'map' a function (e.g.
     'partfrac') onto each term of a very large expression where it
     ordinarily wouldn't be possible to use the function on the entire
     expression due t

### Gini impurity

Another impurity measure is the Gini impurity, which is computed as `gini(ps)`:

In [7]:
gini(ps);

                               ====
                               \
(%o6)                           >      (1 - p) p
                               /
                               ====
                               p in ps

The quantity related to the Gini impurity can be computed as follows:

In [8]:
block(
    [
        fns: ['i, 'X_1, 'X_2, 'Y],
        n: 6,
        gen: lambda([i], [i, random(2), random(2), random(2)]),
        conds: ['X_1, 'X_2, chi('X_1<=0.5), chi('X_2>0.5)],
        target: 'Y,
        data
    ],
    data: build_data(fns, gen, n),
    [
        data, Gini(data, target),
        build_data_from_list(
            ['X, 'Gini[X], 'GiniDrop[X]],
            makelist(
                [X, GiniX(data, target, X), GiniDrop(data, target, X)],
                X, conds
            )
        )
    ]
);

       [ i  X_1  X_2  Y ]
       [                ]
       [ 1   1    0   1 ]
       [                ]
       [ 2   1    1   1 ]
       [                ]  5
(%o7) [[ 3   0    0   0 ], --, 
       [                ]  18
       [ 4   1    0   1 ]
       [                ]
       [ 5   0    1   1 ]
       [                ]
       [ 6   0    0   1 ]
                             [              X               Gini   GiniDrop  ]
                             [                                  X          X ]
                             [                                               ]
                             [                                2       1      ]
                             [             X_1                -       --     ]
                             [                                9       18     ]
                             [                                               ]
                             [                                1       1      ]
                 

where

- `Gini(data, target)` computes the Gini impurity of `target` in `data`.
- `GiniX(data, target, X)` computes the conditional Gini impurity of `target` conditioned on `X`.
- `GiniDrop(data, target, X)` computes the drop in Gini impurity for a splitting criterion `X`.

## Rule-based classifier

### FOIL gain

The following formula computes the FOIL gain 
- from a rule covering `p_0` positives and `n_0` negatives
- to a rule covering `p_1` positives and `n_1` negatives.

In [9]:
foilgain(p_0,n_0,p_1,n_1);

                                                p_1               p_0
(%o8) if equal(p_1, 0) then 0 else p_1 (log2(---------) - log2(---------))
                                             p_1 + n_1         p_0 + n_0

To compute FOIL gain from data:

In [10]:
block(
    [
        fns: ['i, 'X_1, 'X_2, 'Y],
        n: 6,
        gen: lambda([i], [i, random(2), random(2), random(2)]),
        cjts: ['X_1=1, 'X_2=1],
        target: 'Y
    ],
    R: [ar(rest(cjts, -1),target=1), ar(cjts,target=1)],
    data: build_data(fns, gen, n),
    [data, 
    build_data_from_list(
        ["Original rule", "New rule", 'FOILGain],
        [[R[1], R[2], FOILGain(data, target, cjts)]])]
);

       [ i  X_1  X_2  Y ]
       [                ]
       [ 1   1    1   1 ]
       [                ]
       [ 2   0    0   0 ]
       [                ]
(%o9) [[ 3   1    0   1 ], 
       [                ]
       [ 4   0    1   0 ]
       [                ]
       [ 5   1    0   0 ]
       [                ]
       [ 6   1    0   0 ]
       [      Original rule                   New rule              FOILGain ]
       [                                                                     ]]
       [ [ [X_1 = 1]  ⇒  Y = 1 ]  [ [X_1 = 1, X_2 = 1]  ⇒  Y = 1 ]     1     ]

`FOILGain(data, target, cjts)` returns the FOIL gain from rule $R'$ to rule $R$ where
- $R'$: `rest(cjts,-1)` $\implies Y=1$
- $R$: `cjts` $\implies Y=1$

and `rest(cjts,-1)` is the list of conjuncts in `cjts` except the last one.

### FOIL prune

FOIL prune can be computed from the number `p` of positives and the number `n` of negatives covered by a rule.

In [11]:
foilprune(p,n);

                                                      p - n
(%o10)              if equal(p + n, 0) then minf else -----
                                                      p + n

To compute FOIL prune from data:

In [12]:
block(
    [
        fns: ['i, 'X_1, 'X_2, 'Y],
        n: 6,
        gen: lambda([i], [i, random(2), random(2), random(2)]),
        cjts: ['X_1=1, 'X_2=1],
        target: 'Y,
        data
    ],
    R: [ar(cjts,target=1), ar(rest(cjts, -1),target=1)],
    data: build_data(fns, gen, n),
    FP: FOILPrune(data, target, cjts),
    [data, 
    build_data_from_list(
        ["Rule", 'FOILPrune],
        makelist([R[i], FP[i]], i, [1,2]))]
);

        [ i  X_1  X_2  Y ]
        [                ]
        [ 1   0    0   0 ]
        [                ]
        [ 2   1    0   0 ]  [               Rule                FOILPrune ]
        [                ]  [                                             ]
(%o11) [[ 3   1    1   0 ], [ [ [X_1 = 1, X_2 = 1]  ⇒  Y = 1 ]     - 1    ]]
        [                ]  [                                             ]
        [ 4   0    0   0 ]  [     [ [X_1 = 1]  ⇒  Y = 1 ]          - 1    ]
        [                ]
        [ 5   0    0   1 ]
        [                ]
        [ 6   1    1   0 ]

It returns a pair consisting of the FOIL prunes for the rules
- $R$: `cjts` $\implies Y=1$
- $R'$: `rest(cjts,-1)` $\implies Y=1$